In [1]:
import numpy as np
import pymc3 as pm
import pandas as pd

from aesara import tensor as at

You are running the v4 development version of PyMC3 which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3 GitHub branch: https://github.com/pymc-devs/pymc3/tree/v3


In [2]:
K = 47

sunspot_df = pd.read_csv(
    pm.get_data("sunspot.csv"), sep=";", names=["time", "sunspot.year"], usecols=[0, 1]
)
sunspot_df.head()

,time,sunspot.year
0,1700.5,8.3
1,1701.5,18.3
2,1702.5,26.7
3,1703.5,38.3
4,1704.5,60.0


In [3]:
def stick_breaking(beta):
    portion_remaining = at.concatenate([[1], at.cumprod(1 - beta)[:-1]])

    return beta * portion_remaining

Taken from https://nbviewer.jupyter.org/github/fonnesbeck/Bios8366/blob/master/notebooks/Section5_2-Dirichlet-Processes.ipynb

In [4]:
with pm.Model() as model:
    alpha = pm.Gamma("alpha", 1.0, 1.0)
    beta = pm.Beta("beta", 1, alpha, shape=K)
    w = pm.Deterministic("w", stick_breaking(beta))
    # Gamma is conjugate prior to Poisson
    lambda_ = pm.Gamma("lambda_", 300.0, 2.0, shape=K)
    obs = pm.Mixture("obs", w, pm.Poisson.dist(lambda_), observed=sunspot_df["sunspot.year"])

TypeError: dist() takes 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

Taken from https://docs.pymc.io/notebooks/dp_mix.html (code chunk 27)

In [5]:
with pm.Model() as ss_model:
    
    α = pm.Gamma('α', 1., 1.)
    β = pm.Beta('β', 1, α, shape=K)
    w = pm.Deterministic('w', stick_breaking(β))
    
    μ = pm.Uniform('μ', 0., 300., shape=K)
    
    # line below
    obs = pm.Potential('obs', w, pm.Poisson.dist(μ), observed=sunspot_df["sunspot.year"].values)

TypeError: Potential() got an unexpected keyword argument 'observed'